# Imports

In [81]:
from transformers import BertTokenizerFast, BertForTokenClassification, get_scheduler, Trainer, TrainingArguments, AdamW
import torch

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW

from datasets import load_dataset
from datasets import Dataset



# Tokenização com BERT

In [104]:
# Configurações iniciais
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Caminhos para os arquivos de dados
train_file = "./corpus/macmorpho-train.txt"
val_file = "./corpus/macmorpho-dev.txt"
test_file = "./corpus/macmorpho-test.txt"

data_files = {
    "train": train_file,
    "validation": val_file,
    "test": test_file,
}


In [85]:
# Função para carregar os dados
def load_data(filepath):
    sentences = []
    labels = []
    with open(filepath, "r", encoding="utf-8") as file:
        for line in file:
            tokens, tags = [], []
            for pair in line.strip().split():
                word, tag = pair.rsplit("_", 1)
                tokens.append(word)
                tags.append(tag)
            sentences.append(tokens)
            labels.append(tags)
    return sentences, labels

# Carregar os dados
tokens_train, tags_train = load_data(train_file)
tokens_val, tags_val = load_data(val_file)
tokens_test, tags_test = load_data(test_file)

In [86]:
# Inicializar o tokenizer e o modelo pré-treinado
tokenizer = BertTokenizerFast.from_pretrained("neuralmind/bert-base-portuguese-cased")
num_classes = len(set(tag for sent_tags in tags_train for tag in sent_tags))

model = BertForTokenClassification.from_pretrained(
    "neuralmind/bert-base-portuguese-cased",
    num_labels=num_classes
).to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [87]:
# Label Encoder para as tags
tag_encoder = LabelEncoder()
tag_encoder.fit([tag for sent_tags in tags_train for tag in sent_tags])

def preprocess(sentences, tags, tokenizer, tag_encoder):
    input_ids, attention_masks, labels = [], [], []

    for sentence, tag_seq in zip(sentences, tags):
        # Tokenizar as frases
        encoding = tokenizer(
            sentence,
            is_split_into_words=True,
            truncation=True,
            padding="max_length",
            max_length=128,
            return_tensors="pt"
        )

        # Ajustar os rótulos para sub-tokens
        word_ids = encoding.word_ids()
        label_ids = []
        previous_word_idx = None

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Ignorar sub-tokens fora da sequência
            elif word_idx != previous_word_idx:
                label_ids.append(tag_encoder.transform([tag_seq[word_idx]])[0])
            else:
                label_ids.append(-100)  # Ignorar sub-tokens adicionais

            previous_word_idx = word_idx

        input_ids.append(encoding["input_ids"])
        attention_masks.append(encoding["attention_mask"])
        labels.append(torch.tensor(label_ids))

    return torch.cat(input_ids), torch.cat(attention_masks), torch.stack(labels)

In [88]:
# Pré-processar os dados
train_inputs, train_masks, train_labels = preprocess(tokens_train, tags_train, tokenizer, tag_encoder)
val_inputs, val_masks, val_labels = preprocess(tokens_val, tags_val, tokenizer, tag_encoder)

def create_dataloader(inputs, masks, labels, batch_size=16):
    data = TensorDataset(inputs, masks, labels)
    return DataLoader(data, batch_size=batch_size)

In [105]:
# Criar os DataLoaders
train_dataloader = create_dataloader(train_inputs, train_masks, train_labels) #usar para treinar
test_dataloader = create_dataloader(val_inputs, val_masks, val_labels) #usar para testar
val_dataloader = create_dataloader(val_inputs, val_masks, val_labels) #usar para validar

In [90]:
def compute_metrics(eval_pred, unique_labels):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), axis=-1)
    predictions = predictions.flatten().cpu().numpy()
    labels = labels.flatten().cpu().numpy()

    # Filtrando os valores -100, que são usados para ignorar tokens
    mask = labels != -100
    predictions = predictions[mask]
    labels = labels[mask]

    accuracy = accuracy_score(labels, predictions)
    
    # Adicione outras métricas conforme necessário, como F1-score
    return {"accuracy": accuracy}

In [ ]:
PATH = "./training_output"
# Definir o otimizador
optimizer = AdamW(model.parameters(), lr=5e-5)
unique_labels = tag_encoder.classes_

# Supondo que os arquivos estejam no formato linha por linha (ajuste o formato, se necessário)
datasets = load_dataset("text", data_files=data_files)

# Divisão dos datasets
train_dataset = datasets["train"]
test_dataset = datasets["test"]
val_dataset = datasets["validation"]

# Verificação das amostras (opcional)
print(f"Exemplo de treinamento: {train_dataset[0]}")
print(f"Exemplo de teste: {test_dataset[0]}")
print(f"Exemplo de validação: {val_dataset[0]}")

def create_hf_dataset(inputs, masks, labels):
    data = {
        "input_ids": inputs.tolist(),
        "attention_mask": masks.tolist(),
        "labels": labels.tolist(),
    }
    return Dataset.from_dict(data)

# Criar datasets no formato esperado
train_dataset = create_hf_dataset(train_inputs, train_masks, train_labels)
train_dataset = train_dataset.select(range(10000))  # Usar apenas 10.000 exemplos

test_dataset = create_hf_dataset(val_inputs, val_masks, val_labels)
val_dataset = create_hf_dataset(val_inputs, val_masks, val_labels)


training_args = TrainingArguments(
    load_best_model_at_end=True,
    output_dir=PATH,
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=300,
    save_steps=300,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=1,
    logging_dir=f"{PATH}/log",
    save_total_limit=2,
    logging_steps=10,
    fp16=True,
)

# Limpar cache da GPU
torch.cuda.empty_cache()

# Criar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,   # Dataset de treinamento no formato esperado
    eval_dataset=test_dataset,     # Dataset de test no formato esperado
    optimizers=(optimizer, None),  # Passando o otimizador
    compute_metrics=lambda eval_pred: compute_metrics(eval_pred, unique_labels),
)

# Iniciar o treinamento
trainer.train()

Exemplo de treinamento: {'text': 'Jersei_N atinge_V média_N de_PREP Cr$_CUR 1,4_NUM milhão_N na_PREP+ART venda_N da_PREP+ART Pinhal_NPROP em_PREP São_NPROP Paulo_NPROP ._PU'}
Exemplo de validação: {'text': 'Ainda_ADV em_PREP dezembro_N de_PREP 1990_N ,_PU foi_V editada_PCP a_ART famosa_ADJ 289_N ,_PU que_PRO-KS modificava_V a_ART sistemática_N da_PREP+ART arrecadação_N do_PREP+ART ITR_NPROP e_KC alterava_V suas_PROADJ alíquotas_N ._PU'}


Step,Training Loss,Validation Loss


TrainOutput(global_step=156, training_loss=0.4090093519443121, metrics={'train_runtime': 1137.9152, 'train_samples_per_second': 8.788, 'train_steps_per_second': 0.137, 'total_flos': 652338220105728.0, 'train_loss': 0.4090093519443121, 'epoch': 0.9968051118210862})

In [92]:
# Função para salvar o modelo
def save_model(model, tokenizer, save_directory):
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    model.save_pretrained(save_directory)
    tokenizer.save_pretrained(save_directory)
    print(f"Modelo salvo em: {save_directory}")

In [93]:
# Salvar o modelo após o treinamento
save_directory = "./saved_model"
save_model(model, tokenizer, save_directory)

Modelo salvo em: ./saved_model


In [102]:
# Avaliação no conjunto de teste
model.eval()
all_predictions = []
all_labels = []

# Para imprimir valores previstos e esperados
print("Valores previstos e esperados (apenas para os primeiros exemplos):")

# Contador para limitar o número de exemplos impressos
print_count = 0
max_print = 10  # Defina o número de exemplos que você deseja imprimir

with torch.no_grad():
    for batch in val_dataloader:  # Você pode trocar por test_dataloader, se necessário
        batch_input_ids, batch_masks, batch_labels = [b.to(device) for b in batch]
        
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_masks)
        logits = outputs.logits
        
        predictions = torch.argmax(logits, dim=-1).cpu().numpy()
        labels = batch_labels.cpu().numpy()

        # Ignorar tokens -100
        for pred, label, input_ids in zip(predictions, labels, batch_input_ids):
            mask = label != -100
            pred = pred[mask]
            label = label[mask]
            input_ids = input_ids[mask].cpu().numpy()  # Selecionando apenas os ids de tokens relevantes

            all_predictions.extend(pred)
            all_labels.extend(label)

            # Limitar a impressão dos primeiros 10 exemplos
            if print_count < max_print:
                pred_tags = tag_encoder.inverse_transform(pred)  # Convertendo para as tags originais
                label_tags = tag_encoder.inverse_transform(label)  # Convertendo para as tags originais
                sentence = tokenizer.convert_ids_to_tokens(input_ids)  # Usando os input_ids para recuperar a frase original

                print(f"Frase em análise: {' '.join(sentence)}")
                print(f"Previsto: {pred_tags}")
                print(f"Esperado: {label_tags}")
                print("-" * 50)

                print_count += 1
            else:
                break

# Calcular acurácia
accuracy = accuracy_score(all_labels, all_predictions)
print(f"Acurácia no conjunto de teste: {accuracy:.4f}")


Valores previstos e esperados (apenas para os primeiros exemplos):
Frase em análise: Ainda em dezembro de 1990 , foi editada a famosa 28 , que mod a sistemática da arrecadação do I e alter suas al .
Previsto: ['ADV' 'PREP' 'N' 'PREP' 'N' 'PU' 'V' 'PCP' 'ART' 'ADJ' 'N' 'PU' 'PRO-KS'
 'V' 'ART' 'N' 'PREP+ART' 'N' 'PREP+ART' 'NPROP' 'KC' 'V' 'PROADJ' 'N'
 'PU']
Esperado: ['ADV' 'PREP' 'N' 'PREP' 'N' 'PU' 'V' 'PCP' 'ART' 'ADJ' 'N' 'PU' 'PRO-KS'
 'V' 'ART' 'N' 'PREP+ART' 'N' 'PREP+ART' 'NPROP' 'KC' 'V' 'PROADJ' 'N'
 'PU']
--------------------------------------------------
Frase em análise: Porém , como a previsão indica entrada de frente fria , deve - aguarda até quarta e irrig somente se o total de chuva for inferior a 30 mm .
Previsto: ['KC' 'PU' 'KS' 'ART' 'N' 'V' 'N' 'PREP' 'N' 'ADJ' 'PU' 'V' 'PROPESS' 'V'
 'PREP' 'N' 'KC' 'V' 'PDEN' 'KS' 'ART' 'N' 'PREP' 'N' 'V' 'ADJ' 'PREP'
 'NUM' 'N' 'PU']
Esperado: ['KC' 'PU' 'KS' 'ART' 'N' 'V' 'N' 'PREP' 'N' 'ADJ' 'PU' 'V' 'PROPESS' 'V'
 'PREP' 'N'